# Custom negativity intent recognizer(사용자 맞춤 부정적인 의도 인식기).

이 노트북은 우리가 word2vec 모델로 생성한 사용자정의 키워드를 활용하는 Amazon Comprehend의 Custom entities에 대해 학습 데이터셋을 준비하는 방법에 대해 다룹니다. 

우리는 "frustrated"와 의미상으로 유사한 키워드를 기반으로 Custom negativity intent recognizer(사용자 맞춤 부정적인 의도 인식기)를 구현할 것입니다.




In [2]:
# library imports
import re
import numpy as np
import pandas as pd
import matplotlib
import csv


이 예제에서는 통신사 도메인을 위해 전처리와 필터링을 거쳤던, 이전 노트북에서 만들었던 데이터넷을 재사용할 것입니다. 

In [3]:
colnames=['text'] 
tweets = pd.read_csv('./data/tweet_telco.csv',encoding='utf-8',names=colnames, header=None)
print(tweets.shape)
tweets.head()

(32716, 1)


text
0  @sprintcare is the worst customer service | @1...
1  @sprintcare is the worst customer service | @1...
2  @sprintcare is the worst customer service | @1...
3  @115714 y’all lie about your “great” connectio...
4  @115714 whenever I contact customer support, t...

<a id='data-wrangling'></a>

데이터셋을 생성하기 위해서 NEGATIVITY라는 새로운 클래스에 대한 Entity 목록을 제공해야 합니다.

관련된 Entity들을 찾기 위해서 "frustrated"와 의미적으로 유사한 단어를 찾기 위한 커스텀 word2vec 모델을 사용할 것입니다. 
키워드를 생성하는 방법은 blazingtext_word2vec_telco_tweets.ipynb 노트북을 참조하시기 바랍니다 


In [4]:
negative_words = ['Really', 'cheated', 'annoyed', 'unhelpful', 'frustrated', 'upset' , 'unhappy', 'angry', 'badly', 'bad', 'surprised', 'sadly', 'dissatisfied', 'disappointed', 'disgusted']

df_entity_list = pd.DataFrame(negative_words, columns=['Text'])


클래스 레이블과 함께 컬럼을 하나 더 추가합니다. 이것은 Amazon Comprehend의 학습 데이터셋의 요구 사양입니다. 

자세한 내용은 여기를 참조하십시오. 

https://docs.aws.amazon.com/comprehend/latest/dg/cer-entity-list.html

In [5]:
df_entity_list['Type'] = 'NEGATIVE'


학습 파일을 생성합니다. 

In [6]:
tweets['text'].to_csv('./data/raw_negative.csv', encoding='utf-8', index=False)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


In [7]:
!head ./data/raw_negative.csv

"@sprintcare is the worst customer service | @115712 Can you please send us a private message, so that I can gain further details about your account?"
@sprintcare is the worst customer service | @115712 I would love the chance to review the account and provide assistance.
@sprintcare is the worst customer service | @115712 Hello! We never like our customers to feel like they are not valued.
"@115714 y’all lie about your “great” connection. 5 bars LTE, still won’t load something. Smh. | @115713 H there! We'd definitely like to work with you on this, how long have you been experiencing this issue? -AA"
"@115714 whenever I contact customer support, they tell me I have shortcode enabled on my account, but I have never in the 4 years I've tried https://t.co/0G98RtNxPK | @115715 Please send me a private message so that I can send you the link to access your account. -FR"
"@115913 @115911 just called in to switch from AT&amp;T. They wanted $75 to switch 3 phones! I said no way! Inconsistent m

Entity 목록 파일을 생성합니다. 

In [8]:
df_entity_list.to_csv('./data/entity_negative_list.csv', encoding='utf-8', index=False)


In [9]:
!head ./data/entity_negative_list.csv

Text,Type
Really,NEGATIVE
cheated,NEGATIVE
annoyed,NEGATIVE
unhelpful,NEGATIVE
frustrated,NEGATIVE
upset,NEGATIVE
unhappy,NEGATIVE
angry,NEGATIVE
badly,NEGATIVE


원본 telco tweet 데이터셋에서 테스트 파일을 생성합니다. 

In [10]:
tweets['text'].tail(10000).to_csv('./data/telco_device_test.csv', encoding='utf-8', index=False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


## 모델 학습하기

콘솔에서 custom entity recognizer job 을 생성합니다. 상세한 방법은 첫 노트북인 1-AWS-Comprehend-Custom-Entities.ipynb 를 참조하세요.



## Custom entity 모델 테스트하기

Comprehend API를 호출하여 앞서 준비한 테스트 파일에서 Test job을 실행해봅니다. 

In [11]:
!aws comprehend start-entities-detection-job \
     --entity-recognizer-arn "arn:aws:comprehend:us-east-1:{account number}:entity-recognizer/cer-negative" \
     --job-name Test \
     --data-access-role-arn "arn:aws:iam::{account number}:role/service-role/AmazonComprehendServiceRole-cer-test" \
     --language-code en \
     --input-data-config "S3Uri=s3://{bucket name}/custom_entities/telco_device_test.csv" \
     --output-data-config "S3Uri=s3://{bucket name}/custom_entities/output/" \
     --region "us-east-1"

{
    "JobId": "94638d72b858902bed0dedaadaedb104",
    "JobStatus": "SUBMITTED"
}


결과는 --output-data-config 경로에 json 파일이 생성됩니다. 그리고 결과는 Glue와 Athena를 사용하여 확인할 수 있습니다. 

